In [80]:
import os
import pandas as pd
import numpy as np

from itables import show

# Dados do INEP


_O conceito técnico de abandono é diferente de evasão. Abandono quer dizer que o aluno deixa a escola num ano mas retorna no ano seguinte. **Evasão significa que o aluno sai da escola e não volta mais para o sistema.**_


In [81]:
# Constants
INTERIM_DATA_DIR = '../data/interim'
PROCESSED_DATA_DIR = '../data/processed'
PRIMARY_KEY = 'CO_MUNICIPIO'

COMMOM_COLUMNS = [
    "NU_ANO_CENSO",
    "NO_REGIAO",
    "SG_UF",
    "CO_MUNICIPIO",
    "NO_MUNICIPIO",
    "NO_CATEGORIA",
    "NO_DEPENDENCIA"
]
DROP_COLUMNS = [
    "NU_ANO_CENSO",
    "NO_REGIAO",
    "SG_UF",
    "NO_MUNICIPIO",
]
MERGE_COLUMNS = [
    "CO_MUNICIPIO",
    "NO_CATEGORIA",
    "NO_DEPENDENCIA"
]

In [82]:
OBJECT_COLUMNS = [*COMMOM_COLUMNS, "NO_UF"] 
def _cast_columns_to_float(df):
    columns_to_convert = df.select_dtypes(include='object').columns
    columns_to_convert = [col for col in columns_to_convert if col not in OBJECT_COLUMNS]
    df[columns_to_convert] = df[columns_to_convert].astype(float)
    return df

In [83]:
def _add_prefix_header(df, prefix):
    columns_name_map = {}
    columns = df.columns
    for col in columns:
        if col in COMMOM_COLUMNS:
            columns_name_map[col] = col
        else:
            columns_name_map[col] = f"{prefix}_{col}"
    return df.rename(columns=columns_name_map)

In [84]:
def _get_df_info(df, df_name):
    print(f'------------------------ {df_name} ------------------------')
    df.info()
    print('\n--- REGISTROS NÃO REPETIDOS:', 
            df.drop_duplicates().shape[0], '\n')

In [85]:
def _remove_childhood_columns(df):
    # Remove columns with INF in the name - Childhood Education data
    childhood_education_columns = [col_name for col_name in df if "INF" in col_name]
    if childhood_education_columns:
        print("Removing ", ", ".join(childhood_education_columns), "columns")
        df.drop(childhood_education_columns, axis=1, inplace=True)

In [86]:
# Open clean data files
clean_data_dict = {}

years = ["2021"] 
files = os.listdir(os.path.join(INTERIM_DATA_DIR, years[0]))

REMOVE_DATA = ["INSE_clean_2021", "Remuneracao_clean_2020_mock_2021"]

for file in files:
    if file.endswith('.csv') and not (file.rstrip('.csv') in REMOVE_DATA):
        file_path = os.path.join(INTERIM_DATA_DIR, years[0], file)
        print(f'Loading {file_path}')
        
        dataset_name = file.rstrip('.csv')
        clean_data_dict[dataset_name] = pd.read_csv(file_path, delimiter=";")
        
        # Remove columns with INF in the name - Childhood Education data
        _remove_childhood_columns(clean_data_dict[dataset_name])
        # Replace , to .
        clean_data_dict[dataset_name].replace(regex=r',(\d+)', value=r'.\1', inplace=True)
        # Remove "--" string from empty cells
        clean_data_dict[dataset_name].replace("--", np.nan, inplace=True)
        # Cast columns to float
        clean_data_dict[dataset_name] = _cast_columns_to_float(clean_data_dict[dataset_name])
        # Rename columns
        prefix = dataset_name.split('_')[0]
        clean_data_dict[dataset_name] = _add_prefix_header(clean_data_dict[dataset_name], prefix)
                

Loading ../data/interim/2021/TDI_clean_2021.csv
Loading ../data/interim/2021/IED_clean_2021.csv
Loading ../data/interim/2021/TX_clean_2021.csv
Loading ../data/interim/2021/TNR_clean_2021.csv
Loading ../data/interim/2021/ATU_clean_2021.csv
Removing  ED_INF_CAT_0 columns
Loading ../data/interim/2021/IRD_clean_2021.csv
Loading ../data/interim/2021/TXT_transicao_Clean_2021.csv
Loading ../data/interim/2021/ICG_clean_2021.csv
Loading ../data/interim/2021/AFD_clean_2021.csv
Removing  ED_INF_CAT_1, ED_INF_CAT_2, ED_INF_CAT_3, ED_INF_CAT_4, ED_INF_CAT_5 columns


## Verifica tabelas individualmente


In [87]:
# Adequação da Formação Docente
DATASET_NAME = "AFD_clean_2021"
# Categorias:
# 1. Docentes com formação superior de licenciatura na mesma disciplina que lecionam,
#    ou bacharelado na mesma disciplina com curso de complementação pedagógica concluído.
# 2. Docentes com formação superior de bacharelado na disciplina correspondente,
#    mas sem licenciatura ou complementação pedagógica.
# 3. Docentes com licenciatura em área diferente daquela que leciona,
#    ou com bacharelado nas disciplinas da base curricular comum e complementação
#    pedagógica concluída em área diferente daquela que leciona.
# 4. Docentes com outra formação superior não considerada nas categorias anteriores.
# 5. Docentes que não possuem curso superior completo.
# https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2014/docente_formacao_legal/nota_tecnica_indicador_docente_formacao_legal.pdf

clean_data_dict[DATASET_NAME].drop(DROP_COLUMNS, axis=1, inplace=True)

_get_df_info(clean_data_dict[DATASET_NAME], DATASET_NAME)
# show(clean_data_dict[DATASET_NAME])


------------------------ AFD_clean_2021 ------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6992 entries, 0 to 6991
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CO_MUNICIPIO      6992 non-null   int64  
 1   NO_CATEGORIA      6992 non-null   object 
 2   NO_DEPENDENCIA    6992 non-null   object 
 3   AFD_FUN_CAT_1     6372 non-null   float64
 4   AFD_FUN_CAT_2     6372 non-null   float64
 5   AFD_FUN_CAT_3     6372 non-null   float64
 6   AFD_FUN_CAT_4     6372 non-null   float64
 7   AFD_FUN_CAT_5     6372 non-null   float64
 8   AFD_FUN_AI_CAT_1  5393 non-null   float64
 9   AFD_FUN_AI_CAT_2  5393 non-null   float64
 10  AFD_FUN_AI_CAT_3  5393 non-null   float64
 11  AFD_FUN_AI_CAT_4  5393 non-null   float64
 12  AFD_FUN_AI_CAT_5  5393 non-null   float64
 13  AFD_FUN_AF_CAT_1  5608 non-null   float64
 14  AFD_FUN_AF_CAT_2  5608 non-null   float64
 15  AFD_FUN_AF_CAT_3  5608 n

In [88]:
# Média de Alunos por Turma
DATASET_NAME = "ATU_clean_2021"
clean_data_dict[DATASET_NAME].drop(DROP_COLUMNS, axis=1, inplace=True)

_get_df_info(clean_data_dict[DATASET_NAME], DATASET_NAME)
# show(clean_data_dict[DATASET_NAME])

------------------------ ATU_clean_2021 ------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CO_MUNICIPIO    6941 non-null   int64  
 1   NO_CATEGORIA    6941 non-null   object 
 2   NO_DEPENDENCIA  6941 non-null   object 
 3   ATU_FUN_CAT_0   6307 non-null   float64
 4   ATU_MED_CAT_0   4919 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 271.3+ KB

--- REGISTROS NÃO REPETIDOS: 6941 



In [89]:
# Complexidade de Gestão da Escola
DATASET_NAME = "ICG_clean_2021"
# Descrição dos níveis de complexidade de gestão:
# Nível 1: Porte inferior a 50 matrículas, operando em único turno e etapa e apresentando a
# Educação Infantil ou Anos Iniciais como etapa mais elevada*.
# Nível 2: Porte entre 50 e 300 matrículas, operando em 2 turnos, com oferta de até 2 etapas e
# apresentando a Educação Infantil ou Anos Iniciais como etapa mais elevada*.
# Nível 3: Porte entre 50 e 500 matrículas, operando em 2 turnos, com 2 ou 3 etapas e
# apresentando os Anos Finais como etapa mais elevada*.
# Nível 4: Porte entre 150 e 1000 matrículas, operando em 2 ou 3 turnos, com 2 ou 3 etapas,
# apresentando Ensino Médio/profissional ou a EJA como etapa mais elevada*.
# Nível 5: Porte entre 150 e 1000 matrículas, operando em 3 turnos, com 2 ou 3 etapas,
# apresentando a EJA como etapa mais elevada*.
# Nível 6: Porte superior à 500 matrículas, operando em 3 turnos, com 4 ou mais etapas,
# apresentando a EJA como etapa mais elevada*.
# https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2014/escola_complexidade_gestao/nota_tecnica_indicador_escola_complexidade_gestao.pdf

clean_data_dict[DATASET_NAME].drop(DROP_COLUMNS, axis=1, inplace=True)

_get_df_info(clean_data_dict[DATASET_NAME], DATASET_NAME)
# show(clean_data_dict[DATASET_NAME])

------------------------ ICG_clean_2021 ------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7009 entries, 0 to 7008
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CO_MUNICIPIO       7009 non-null   int64  
 1   NO_CATEGORIA       7009 non-null   object 
 2   NO_DEPENDENCIA     7009 non-null   object 
 3   ICG_EDU_BAS_CAT_1  7009 non-null   float64
 4   ICG_EDU_BAS_CAT_2  7009 non-null   float64
 5   ICG_EDU_BAS_CAT_3  7009 non-null   float64
 6   ICG_EDU_BAS_CAT_4  7009 non-null   float64
 7   ICG_EDU_BAS_CAT_5  7009 non-null   float64
 8   ICG_EDU_BAS_CAT_6  7009 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 492.9+ KB

--- REGISTROS NÃO REPETIDOS: 7009 



In [90]:
# Esforço Docente
DATASET_NAME = "IED_clean_2021"
# Descrição dos níveis de esforço docente
# Nível 1: Docente que tem até 25 alunos e atua em um único turno, escola e etapa.
# Nível 2: Docente que tem entre 25 e 150 alunos e atua em um único turno, escola e etapa.
# Nível 3: Docente que tem entre 25 e 300 alunos e atua em um ou dois turnos em uma única escola e etapa.
# Nível 4: Docente que tem entre 50 e 400 alunos e atua em dois turnos, em uma ou duas escolas e em duas etapas.
# Nível 5: Docente que tem mais de 300 alunos e atua nos três turnos, em duas ou três escolas e em duas etapas ou três etapas.
# Nível 6: Docente que tem mais de 400 alunos e atua nos três turnos, em duas ou três escolas e em duas etapas ou três etapas.
#https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2014/docente_esforco/nota_tecnica_indicador_docente_esforco.pdf

clean_data_dict[DATASET_NAME].drop(DROP_COLUMNS, axis=1, inplace=True)

_get_df_info(clean_data_dict[DATASET_NAME], DATASET_NAME)
# show(clean_data_dict[DATASET_NAME])

------------------------ IED_clean_2021 ------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6775 entries, 0 to 6774
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CO_MUNICIPIO      6775 non-null   int64  
 1   NO_CATEGORIA      6775 non-null   object 
 2   NO_DEPENDENCIA    6775 non-null   object 
 3   IED_FUN_CAT_1     6376 non-null   float64
 4   IED_FUN_CAT_2     6376 non-null   float64
 5   IED_FUN_CAT_3     6376 non-null   float64
 6   IED_FUN_CAT_4     6376 non-null   float64
 7   IED_FUN_CAT_5     6376 non-null   float64
 8   IED_FUN_CAT_6     6376 non-null   float64
 9   IED_FUN_AI_CAT_1  5395 non-null   float64
 10  IED_FUN_AI_CAT_2  5395 non-null   float64
 11  IED_FUN_AI_CAT_3  5395 non-null   float64
 12  IED_FUN_AI_CAT_4  5395 non-null   float64
 13  IED_FUN_AI_CAT_5  5395 non-null   float64
 14  IED_FUN_AI_CAT_6  5395 non-null   float64
 15  IED_FUN_AF_CAT_1  5615 n

In [91]:
# #TODO: O dado é importante mas vamos realmente usar? So tem 2011-2013, 2015, 2019, 2021
 
# # Nível Socioeconômico
# DATASET_NAME = "INSE_clean_2021"
# #https://download.inep.gov.br/areas_de_atuacao/Indicadores_de_nivel_Nota_tenica_2021.pdf

# drop_columns_inse = [*DROP_COLUMNS, "CO_UF", "NO_UF"]
# drop_columns_inse.remove("NO_REGIAO")

# clean_data_dict[DATASET_NAME].drop(drop_columns_inse, axis=1, inplace=True)

# _get_df_info(clean_data_dict[DATASET_NAME], DATASET_NAME)
# # show(clean_data_dict[DATASET_NAME])

In [92]:
# Regularidade do Corpo Docente
DATASET_NAME = "IRD_clean_2021"

clean_data_dict[DATASET_NAME].drop(DROP_COLUMNS, axis=1, inplace=True)

_get_df_info(clean_data_dict[DATASET_NAME], DATASET_NAME)
# show(clean_data_dict[DATASET_NAME])

------------------------ IRD_clean_2021 ------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6962 entries, 0 to 6961
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CO_MUNICIPIO       6962 non-null   int64  
 1   NO_CATEGORIA       6962 non-null   object 
 2   NO_DEPENDENCIA     6962 non-null   object 
 3   IRD_EDU_BAS_CAT_1  6962 non-null   float64
 4   IRD_EDU_BAS_CAT_2  6962 non-null   float64
 5   IRD_EDU_BAS_CAT_3  6962 non-null   float64
 6   IRD_EDU_BAS_CAT_4  6962 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 380.9+ KB

--- REGISTROS NÃO REPETIDOS: 6962 



In [93]:
# #TODO: vamos mesmo usar? Não temos 2021 e provavelmente não teremos até o final do trabalho
# Talvez buscar um dado parecido em outra fonte

# # Remuneração Média dos Docentes
# DATASET_NAME = "Remuneracao_clean_2020_mock_2021"

# clean_data_dict[DATASET_NAME].drop(DROP_COLUMNS, axis=1, inplace=True)
# print(clean_data_dict[DATASET_NAME].dtypes)

# _get_df_info(clean_data_dict[DATASET_NAME], DATASET_NAME)
# show(clean_data_dict[DATASET_NAME])

In [94]:
# Taxas de Distorção Idade-série
DATASET_NAME = "TDI_clean_2021"

clean_data_dict[DATASET_NAME].drop(DROP_COLUMNS, axis=1, inplace=True)

# TDI_MED_04_CAT_0 is the `4ª Série` column in the in the `Taxas de Distorção Idade-série` table
# This column has 97.2% of missing values, because few schools have `4ª Série`. So we drop to remove it.
clean_data_dict[DATASET_NAME].drop(["TDI_MED_04_CAT_0"], axis=1, inplace=True)

_get_df_info(clean_data_dict[DATASET_NAME], DATASET_NAME)
# show(clean_data_dict[DATASET_NAME])

------------------------ TDI_clean_2021 ------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6706 entries, 0 to 6705
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CO_MUNICIPIO      6706 non-null   int64  
 1   NO_CATEGORIA      6706 non-null   object 
 2   NO_DEPENDENCIA    6706 non-null   object 
 3   TDI_FUN_CAT_0     6307 non-null   float64
 4   TDI_FUN_AI_CAT_0  5479 non-null   float64
 5   TDI_FUN_AF_CAT_0  5243 non-null   float64
 6   TDI_FUN_01_CAT_0  5319 non-null   float64
 7   TDI_FUN_02_CAT_0  5376 non-null   float64
 8   TDI_FUN_03_CAT_0  5388 non-null   float64
 9   TDI_FUN_04_CAT_0  5377 non-null   float64
 10  TDI_FUN_05_CAT_0  5384 non-null   float64
 11  TDI_FUN_06_CAT_0  5239 non-null   float64
 12  TDI_FUN_07_CAT_0  5234 non-null   float64
 13  TDI_FUN_08_CAT_0  5220 non-null   float64
 14  TDI_FUN_09_CAT_0  5223 non-null   float64
 15  TDI_MED_CAT_0     4911 n

In [95]:
# Taxas de Não-resposta
DATASET_NAME = "TNR_clean_2021"

clean_data_dict[DATASET_NAME].drop(DROP_COLUMNS, axis=1, inplace=True)

_get_df_info(clean_data_dict[DATASET_NAME], DATASET_NAME)
# show(clean_data_dict[DATASET_NAME])

------------------------ TNR_clean_2021 ------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6708 entries, 0 to 6707
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CO_MUNICIPIO    6708 non-null   int64  
 1   NO_CATEGORIA    6708 non-null   object 
 2   NO_DEPENDENCIA  6708 non-null   object 
 3   TNR_4_CAT_FUN   6309 non-null   float64
 4   TNR_4_CAT_MED   4919 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 262.2+ KB

--- REGISTROS NÃO REPETIDOS: 6708 



In [96]:
# Taxas de Rendimento Escolar
DATASET_NAME = "TX_clean_2021"

clean_data_dict[DATASET_NAME].drop(DROP_COLUMNS, axis=1, inplace=True)

_get_df_info(clean_data_dict[DATASET_NAME], DATASET_NAME)
# show(clean_data_dict[DATASET_NAME])

------------------------ TX_clean_2021 ------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6708 entries, 0 to 6707
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CO_MUNICIPIO    6708 non-null   int64  
 1   NO_CATEGORIA    6708 non-null   object 
 2   NO_DEPENDENCIA  6708 non-null   object 
 3   TX_1_CAT_FUN    6305 non-null   float64
 4   TX_1_CAT_MED    4913 non-null   float64
 5   TX_2_CAT_FUN    6305 non-null   float64
 6   TX_2_CAT_MED    4913 non-null   float64
 7   TX_3_CAT_FUN    6305 non-null   float64
 8   TX_3_CAT_MED    4913 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 471.8+ KB

--- REGISTROS NÃO REPETIDOS: 6708 



In [97]:
# Taxas de Transição
DATASET_NAME = "TXT_transicao_Clean_2021"

drop_columns_tx = [*DROP_COLUMNS, "TXT_NO_UF"]
drop_columns_tx.remove("SG_UF")
clean_data_dict[DATASET_NAME].drop(drop_columns_tx, axis=1, inplace=True)

_get_df_info(clean_data_dict[DATASET_NAME], DATASET_NAME)
# show(clean_data_dict[DATASET_NAME])

------------------------ TXT_transicao_Clean_2021 ------------------------


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2513 entries, 0 to 2512
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CO_MUNICIPIO       2513 non-null   int64  
 1   NO_CATEGORIA       2513 non-null   object 
 2   NO_DEPENDENCIA     2513 non-null   object 
 3   TXT_1_CAT1_CATFUN  2497 non-null   float64
 4   TXT_1_CAT1_CATMED  2338 non-null   float64
 5   TXT_1_CAT2_CATFUN  2497 non-null   float64
 6   TXT_1_CAT2_CATMED  2338 non-null   float64
 7   TXT_1_CAT3_CATFUN  2497 non-null   float64
 8   TXT_1_CAT3_CATMED  2338 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 176.8+ KB

--- REGISTROS NÃO REPETIDOS: 2513 



# Dados IBGE


In [98]:
import requests
from functools import reduce

In [99]:
def _get_county_pib_df(json_data):
    pib_municipios_ibge_json = json_data[0]["resultados"][0]["series"]
    pib_municipios_ibge_df = pd.json_normalize(pib_municipios_ibge_json)
    pib_municipios_ibge_df = pib_municipios_ibge_df.drop(['localidade.nivel.id', 'localidade.nivel.nome', "localidade.nome"], axis=1)
    return pib_municipios_ibge_df

In [100]:
# Build the table with county PIB

# years = list(range(2013, 2022))
years = [2021]
counties_pib_dfs = []

for year in years:
    print(f"Getting data for year {year}")
    county_pib_url = f"https://servicodados.ibge.gov.br/api/v3/agregados/5938/periodos/{year}/variaveis/37?localidades=N6[all]"
    response = requests.get(county_pib_url)

    if response.status_code == 200:
        data = response.json()
        print(f"Building IBGE {year} dataframe...")
        counties_pib_dfs.append(_get_county_pib_df(data))
    else:
        print("Request failed with status code:", response.status_code)

print("Merging IBGE dataframes...")
complete_counties_pib = reduce(lambda df1,df2: pd.merge(df1,df2,on='localidade.id', how='outer'), counties_pib_dfs)

# Rename Columns
new_column_names = {'serie.{}'.format(year): 'pib_{}'.format(year) for year in years}
new_column_names["localidade.id"] = PRIMARY_KEY
complete_counties_pib = complete_counties_pib.rename(columns=new_column_names)

# Cast columns to int
complete_counties_pib = complete_counties_pib.astype(int)

print("Finished building the table with county PIB")

Getting data for year 2021


Building IBGE 2021 dataframe...
Merging IBGE dataframes...
Finished building the table with county PIB


# Merge


In [101]:
TARGET_DATASET = "TXT_transicao_Clean_2021"
total_colunas = clean_data_dict[TARGET_DATASET].shape[1]
# Renomeia colunas para facilitar manipulação
clean_data_dict[TARGET_DATASET].rename(columns={
    'TXT_1_CAT3_CATFUN': 'EVASAO_FUN', 
    'TXT_1_CAT3_CATMED': 'EVASAO_MED'
    }, inplace=True)

# Todas as linhas possui informação de evasão em pelo menos umas colunas EVASAO_FUN ou EVASAO_MED
print(
    "Número de linhas com EVASAO_FUN e EVASAO_MED como NaN:",
    clean_data_dict[TARGET_DATASET].query("@pd.isna(EVASAO_FUN) & @pd.isna(EVASAO_MED)").shape[0]
)

Número de linhas com EVASAO_FUN e EVASAO_MED como NaN: 0


In [102]:
merged_data = clean_data_dict[TARGET_DATASET] # use TXT_transicao_Clean_2021 as base

for key, data in clean_data_dict.items():
    if key != TARGET_DATASET:
        print(f"Merging {key}...")
        total_colunas += data.shape[1] - len(MERGE_COLUMNS)
        print(f">>>{key} add {data.shape[1] - len(MERGE_COLUMNS)} new features")
        merged_data = pd.merge(merged_data, data, on=MERGE_COLUMNS, how='left')

# Add IBGE data
print(f"Merging IBGE_data...")
total_colunas+=1
print(f">>>IBGE_data add 1 new features")
merged_data = pd.merge(merged_data, complete_counties_pib, on=PRIMARY_KEY, how='left')

print(f"Total de colunas: {total_colunas}")

Merging TDI_clean_2021...
>>>TDI_clean_2021 add 16 new features
Merging IED_clean_2021...
>>>IED_clean_2021 add 24 new features
Merging TX_clean_2021...
>>>TX_clean_2021 add 6 new features
Merging TNR_clean_2021...
>>>TNR_clean_2021 add 2 new features
Merging ATU_clean_2021...
>>>ATU_clean_2021 add 2 new features
Merging IRD_clean_2021...
>>>IRD_clean_2021 add 4 new features
Merging ICG_clean_2021...
>>>ICG_clean_2021 add 6 new features
Merging AFD_clean_2021...
>>>AFD_clean_2021 add 20 new features
Merging IBGE_data...
>>>IBGE_data add 1 new features
Total de colunas: 90


In [ ]:
merged_data.info()
# show(merged_data)

In [104]:
# Save data
merged_data.to_csv(F'{PROCESSED_DATA_DIR}/complete_data.csv', sep=';', index=False)

In [105]:
# Save sliced data: NO_CATEGORIA==Total and NO_DEPENDENCIA==Total
total_total_data = merged_data[
    (merged_data['NO_CATEGORIA'] == 'Total') & (merged_data['NO_DEPENDENCIA'] == 'Total')
].copy()
total_total_data.drop(['NO_CATEGORIA', 'NO_DEPENDENCIA'], axis=1, inplace=True)

total_total_data.to_csv(F'{PROCESSED_DATA_DIR}/total_total_data.csv', sep=';', index=False)

In [108]:
# Save sliced data: Onle data without FUN columns
total_total_data_med = total_total_data.filter(regex='^(?!.*FUN)')
total_total_data_med.to_csv(F'{PROCESSED_DATA_DIR}/total_total_data_med.csv', sep=';', index=False)

In [109]:
# Save sliced data: Onle data without MED columns
total_total_data_fun = total_total_data.filter(regex='^(?!.*MED)')
total_total_data_fun.to_csv(F'{PROCESSED_DATA_DIR}/total_total_data_fun.csv', sep=';', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 645 entries, 0 to 2509
Data columns (total 65 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CO_MUNICIPIO       645 non-null    int64  
 1   TXT_1_CAT1_CATFUN  645 non-null    float64
 2   TXT_1_CAT2_CATFUN  645 non-null    float64
 3   EVASAO_FUN         645 non-null    float64
 4   TDI_FUN_CAT_0      645 non-null    float64
 5   TDI_FUN_AI_CAT_0   645 non-null    float64
 6   TDI_FUN_AF_CAT_0   645 non-null    float64
 7   TDI_FUN_01_CAT_0   645 non-null    float64
 8   TDI_FUN_02_CAT_0   645 non-null    float64
 9   TDI_FUN_03_CAT_0   645 non-null    float64
 10  TDI_FUN_04_CAT_0   645 non-null    float64
 11  TDI_FUN_05_CAT_0   645 non-null    float64
 12  TDI_FUN_06_CAT_0   645 non-null    float64
 13  TDI_FUN_07_CAT_0   645 non-null    float64
 14  TDI_FUN_08_CAT_0   645 non-null    float64
 15  TDI_FUN_09_CAT_0   645 non-null    float64
 16  IED_FUN_CAT_1      645 non-nul